# About 

This Colab notebook train different types of CNN classififers on the MNIST dataset that will be used in the adversarial machine learning experiments. Note that this script does not include any adversarial aspect. The neural networks are simply trained on the MNIST dataset.  


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Git/ProtoShotXAI-1/

/content/gdrive/MyDrive/Git/ProtoShotXAI-1


# Imports, Program Constants and Data Preparation 

This section of the notebook prepares the MNIST dataset. We force all the networks to use an `input_shape=(28,28,1)`. 

In [3]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model

from keras.datasets import mnist

In [4]:
# shape of the mnist dataset 
input_shape = (28, 28, 1)
# number of classes 
num_classes = 10

In [5]:
# load the mnist dataset
mnist = tf.keras.datasets.mnist

# scale the data 
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# reshape the datasets 
# x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
# x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))
x_train = np.expand_dims(x_train,axis = 3)/255
x_test = np.expand_dims(x_test,axis=3)/255


# convert the outputs to one-hot-encodings 
y_train_oh = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_oh = tf.keras.utils.to_categorical(y_test, num_classes)

11501568/11490434 [==============================] - 0s 0us/step


# Standard CNN

In [ ]:
num_epoch = 25
batch_size = 512
verbose = 1

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss_fn, metrics=['acc'])
model.fit(x_train, y_train_oh, epochs=num_epoch, batch_size=batch_size)
model.save('models/mnist_cnn.h5')
print('CNN Accuracy: %f' % ((np.argmax(model.predict(x_test), axis=1) == y_test).sum()/len(y_test)*100))

# VGG16

In [ ]:
model = Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(28, 28, 1)), 
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'), 
    tf.keras.layers.MaxPooling2D(2, 2), 
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'), 
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'), 
    tf.keras.layers.MaxPooling2D(2, 2), 
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2), 
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2, 1),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(4096, activation='relu'), 
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(4096, activation='relu'), 
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(10, activation='softmax')
])

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss_fn, metrics=['acc'])
model.fit(x_train, y_train_oh, epochs=num_epoch, batch_size=batch_size)
model.save('models/mnist_vgg16.h5')

print('CNN Accuracy: %f' % ((np.argmax(model.predict(x_test), axis=1) == y_test).sum()/len(y_test)*100))

# VGG Variation

In [7]:
num_epoch = 25
batch_size = 512
verbose = 1

model = Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(28, 28, 1)), 
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'), 
    tf.keras.layers.MaxPooling2D(2, 2), 
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'), 
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'), 
    tf.keras.layers.MaxPooling2D(2, 2), 
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2), 
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2, 1),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(4096, activation='relu'), 
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(2048, activation='relu'), 
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(10, activation='softmax')
])

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss_fn, metrics=['acc'])
model.fit(x_train, y_train_oh, epochs=num_epoch, batch_size=batch_size)
model.save('models/mnist_vgg16_v2.h5')

print('CNN Accuracy: %f' % ((np.argmax(model.predict(x_test), axis=1) == y_test).sum()/len(y_test)*100))

Epoch 1/25
118/118 [==============================] - 17s 44ms/step - loss: 0.6879 - acc: 0.7771
Epoch 2/25
118/118 [==============================] - 5s 41ms/step - loss: 0.1234 - acc: 0.9720
Epoch 3/25
118/118 [==============================] - 5s 41ms/step - loss: 0.0868 - acc: 0.9823
Epoch 4/25
118/118 [==============================] - 5s 41ms/step - loss: 0.0674 - acc: 0.9866
Epoch 5/25
118/118 [==============================] - 5s 41ms/step - loss: 0.0620 - acc: 0.9873
Epoch 6/25
118/118 [==============================] - 5s 41ms/step - loss: 0.0492 - acc: 0.9901
Epoch 7/25
118/118 [==============================] - 5s 41ms/step - loss: 0.0481 - acc: 0.9900
Epoch 8/25
118/118 [==============================] - 5s 41ms/step - loss: 0.0413 - acc: 0.9913
Epoch 9/25
118/118 [==============================] - 5s 41ms/step - loss: 0.0398 - acc: 0.9923
Epoch 10/25
118/118 [==============================] - 5s 41ms/step - loss: 0.0329 - acc: 0.9937
Epoch 11/25
118/118 [=================